In [2]:
import keras
from keras.applications.inception_resnet_v2 import InceptionResNetV2
from keras.preprocessing import image
from keras.engine import Layer
from keras.layers import Conv2D, UpSampling2D, InputLayer, MaxPool2D, Input, Reshape, merge, concatenate
from keras.layers import Activation,Dropout
from keras.callbacks import TensorBoard
from keras.models import Sequential, Model
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from keras.layers.core import RepeatVector
from keras.applications.inception_resnet_v2 import preprocess_input
from skimage.color import rgb2lab, lab2rgb, rgb2gray, gray2rgb
from skimage.transform import resize
from skimage.io import imsave
import numpy as np
import os
import tensorflow as tf
from PIL import Image, ImageFile

Using TensorFlow backend.


In [4]:
from google.colab import drive
drive.mount('/16wh1a0596@bvrithyderabad.edu.in')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /16wh1a0596@bvrithyderabad.edu.in


In [0]:
# Get images
X = []
for filename in os.listdir('/16wh1a0596@bvrithyderabad.edu.in/My Drive/Colab Notebooks/direc/TrainSet/'):
    X.append(img_to_array(load_img('/16wh1a0596@bvrithyderabad.edu.in/My Drive/Colab Notebooks/direc/TrainSet/'+filename)))
X = np.array(X, dtype=float)
split = int(0.95*len(X))
Xtrain = X[:split]
Xtrain = 1.0/255*Xtrain

In [0]:
#Load weights
inception = InceptionResNetV2(weights=None, include_top=True)
inception.load_weights('/16wh1a0596@bvrithyderabad.edu.in/My Drive/Colab Notebooks/direc/inception_resnet_v2_weights_tf_dim_ordering_tf_kernels.h5')
inception.graph = tf.get_default_graph()


In [0]:
encoder_input = Input(shape=(256, 256, 1,))
encoder_output = Conv2D(32,(3,3),activation='relu',padding='same')(encoder_input)
encoder_output = MaxPool2D(pool_size=(2,2),strides=2)(encoder_input)
encoder_output = Conv2D(32, (3,3), activation='relu', padding='same')(encoder_output)
encoder_output = MaxPool2D(pool_size=(2,2),strides=2)(encoder_output)
encoder_output = Conv2D(32, (3,3), activation='relu', padding='same')(encoder_output)
encoder_output = MaxPool2D(pool_size=(3,3),strides=2,padding='same')(encoder_output)
print(encoder_output)

Tensor("max_pooling2d_40/MaxPool:0", shape=(?, 32, 32, 32), dtype=float32)


In [0]:
embed_input = Input(shape=(32,))
fusion_output = RepeatVector(32 * 32)(embed_input) 
fusion_output = Reshape(([32, 32, 32]))(fusion_output)
fusion_output = concatenate([encoder_output, fusion_output], axis=3) 
print(fusion_output)
fusion_output = Conv2D(256, (1, 1), activation='relu', padding='same')(fusion_output)

Tensor("concatenate_20/concat:0", shape=(?, 32, 32, 64), dtype=float32)


In [0]:
#Decoder
decoder_output = Conv2D(32, (3,3), activation='relu', padding='same')(fusion_output)
print(decoder_output)
decoder_output = UpSampling2D((2, 2))(decoder_output)
decoder_output = Conv2D(32, (3,3), activation='relu', padding='same')(decoder_output)
decoder_output = UpSampling2D((2, 2))(decoder_output)
decoder_output = Conv2D(32, (3,3), activation='relu', padding='same')(decoder_output)
decoder_output = Conv2D(2, (3, 3), activation='tanh', padding='same')(decoder_output)
decoder_output = UpSampling2D((2, 2))(decoder_output)


Tensor("conv2d_325/Relu:0", shape=(?, 32, 32, 32), dtype=float32)


In [0]:
model = Model(inputs=[encoder_input, embed_input], outputs=decoder_output)

In [0]:
#Create embedding
def create_inception_embedding(grayscaled_rgb):
    grayscaled_rgb_resized = []
    for i in grayscaled_rgb:
        i = resize(i, (299, 299, 3), mode='constant')
        grayscaled_rgb_resized.append(i)
    grayscaled_rgb_resized = np.array(grayscaled_rgb_resized)
    grayscaled_rgb_resized = preprocess_input(grayscaled_rgb_resized)
    with inception.graph.as_default():
        embed = inception.predict(grayscaled_rgb_resized)
    return embed

In [0]:
datagen = ImageDataGenerator(
        shear_range=0.4,
        zoom_range=0.4,
        rotation_range=40,
        horizontal_flip=True)

In [0]:
#Generate training data
batch_size = 20
def image_a_b_gen(batch_size):
    for batch in datagen.flow(Xtrain, batch_size=batch_size):
        grayscaled_rgb = gray2rgb(rgb2gray(batch))
        embed = create_inception_embedding(grayscaled_rgb)
        lab_batch = rgb2lab(batch)
        X_batch = lab_batch[:,:,:,0]
        X_batch = X_batch.reshape(X_batch.shape+(1,))
        Y_batch = lab_batch[:,:,:,1:] / 128
        yield ([X_batch, create_inception_embedding(grayscaled_rgb)], Y_batch)

In [1]:
#Train model      
tensorboard = TensorBoard(log_dir="/output")
model.compile(optimizer='adam', loss='mse',metrics=['accuracy'])
model.fit(image_a_b_gen(batch_size), epochs=50, steps_per_epoch=1)

NameError: ignored

In [0]:
Xtest = rgb2lab(1.0/255*X[split:])[:,:,:,0]
Xtest = Xtest.reshape(Xtest.shape+(1,))

color_me_embed = create_inception_embedding(Xtest)
output = model.predict([Xtest, color_me_embed])
output = output * 128

# Output colorizations
for i in range(len(output)):
    cur = np.zeros((256, 256, 3))
    cur[:,:,0] = Xtest[i][:,:,0]
    cur[:,:,1:] = output[i]
    imsave("resultSet/"+str(i)+".png", lab2rgb(cur))